In [30]:
from telethon.sync import TelegramClient
from client import  parse_channel_messages
from config import API_ID, API_HASH, TOKEN

In [31]:
client = TelegramClient('session_name', API_ID, API_HASH)

In [32]:
ans = await parse_channel_messages(client, 'cbrstocks', limit=3)

In [33]:
ans

['#GLTR #Дивиденды\n**Выплата дивидендов в фокусе Globaltrans**, но пока приостановлена из-за технических сложностей. Работаем над решением — компания\n\nПосле успешного завершения редомициляции в ADGM текущей задачей является создание полноценной финансовой инфраструктуры в Абу-Даби — Globaltrans',
 '#GLTR #Отчетность\n**🖨**** Globaltrans: МСФО 2023**\n\nЧистая прибыль 38.6 млрд руб (+55% г/г)',
 '❗️ **FIVE, VKCO и другие — трейды, которые вы упустили на прошлой неделе**\n\nПодборка быстрых трейдов, которые отторговали пользователи Шпиона РЦБ\n\n• FIVE до +12% на шорте после новостей о приостановке торгов\n• VKCO +3% после идеи во влиятельном канале\n• SBER +1.5% на словах Мишустина\n\nИ это только часть сигналов, на которых вы могли заработать со Шпионом. \n\nНачните новую торговую неделю с преимуществом над большинством трейдеров. \n\n👉 [Подключиться к Шпиону](https://t.me/stockspy_bot)']